---
title: "Emergent Forces: Vortex Interaction"
author: "Raúl Chiclano"
date: "2025-11-30"
categories: [matter, forces]
description: "How hydrodynamic forces mimic particle interactions."
format:
  html:
    code-fold: true
execute:
  freeze: true
---

We initialize two vortices with the same topological charge ($Q=1$). According to the hydrodynamic model, they should interact via the phase field, orbiting each other without merging.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# 1. CONFIGURACIÓN
N = 128
L = 30.0
dt = 0.005
dt_im = 0.005
g = 2.0
rho_0 = 1.0

x = np.linspace(-L/2, L/2, N)
y = np.linspace(-L/2, L/2, N)
X, Y = np.meshgrid(x, y)
dx = x[1] - x[0]
k = 2 * np.pi * np.fft.fftfreq(N, d=dx)
KX, KY = np.meshgrid(k, k)
K2 = KX**2 + KY**2

# Potencial de trampa suave
R_grid = np.sqrt(X**2 + Y**2)
V_trap = 0.5 * (R_grid / (0.45*L))**10
V_trap[V_trap > 100] = 100

# 2. CONDICIÓN INICIAL (DOS VÓRTICES)
d_sep = 3.0
x1, y1 = -d_sep, 0.0
x2, y2 = +d_sep, 0.0

X1, Y1 = X - x1, Y - y1
R1, Theta1 = np.sqrt(X1**2 + Y1**2), np.arctan2(Y1, X1)

X2, Y2 = X - x2, Y - y2
R2, Theta2 = np.sqrt(X2**2 + Y2**2), np.arctan2(Y2, X2)

xi = 1.0 / np.sqrt(g * rho_0)
xi_scale = 2.0 * xi

# Ansatz de producto
Psi = np.sqrt(rho_0) * np.tanh(R1/xi_scale) * np.tanh(R2/xi_scale) * np.exp(1j * (Theta1 + Theta2))
Theta_ref = np.angle(np.exp(1j * (Theta1 + Theta2)))

# 3. RELAJACIÓN (TIEMPO IMAGINARIO)
U_kin_im = np.exp(-(K2 / 2.0) * dt_im)

for i in range(100):
    density = np.abs(Psi)**2
    Psi = Psi * np.exp(-(V_trap + g * density) * (dt_im / 2))
    Psi_k = np.fft.fft2(Psi)
    Psi_k *= U_kin_im
    Psi = np.fft.ifft2(Psi_k)
    density = np.abs(Psi)**2
    Psi = Psi * np.exp(-(V_trap + g * density) * (dt_im / 2))
    norm_factor = np.sqrt(rho_0) / np.max(np.abs(Psi))
    Psi *= norm_factor
    Psi = np.abs(Psi) * np.exp(1j * Theta_ref)

# 4. EVOLUCIÓN REAL
U_kin_real = np.exp(-1j * (K2 / 2.0) * dt)

def evolution_step_real(psi_in):
    psi_mod = psi_in * np.exp(-1j * (V_trap + g * np.abs(psi_in)**2) * (dt / 2))
    psi_k = np.fft.fft2(psi_mod)
    psi_k *= U_kin_real
    psi_mod = np.fft.ifft2(psi_k)
    psi_out = psi_mod * np.exp(-1j * (V_trap + g * np.abs(psi_mod)**2) * (dt / 2))
    return psi_out

# Animación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
plt.close()

im1 = ax1.imshow(np.abs(Psi)**2, extent=[-L/2, L/2, -L/2, L/2], cmap='inferno')
ax1.set_title("Density |Psi|^2")
im2 = ax2.imshow(np.angle(Psi), extent=[-L/2, L/2, -L/2, L/2], cmap='hsv')
ax2.set_title("Phase Theta")

def update(frame):
    global Psi
    for _ in range(5):
        Psi = evolution_step_real(Psi)
    im1.set_data(np.abs(Psi)**2)
    im2.set_data(np.angle(Psi))
    return im1, im2

ani = animation.FuncAnimation(fig, update, frames=80, interval=50, blit=True)
display(HTML(ani.to_html5_video()))

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '1000x500', '-pix_fmt', 'rgba', '-framerate', '20.0', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', '/tmp/tmpp7xm72g0/temp.m4v']' returned non-zero exit status 255.